# Main Function

In [14]:
from io import BytesIO
import numpy as np
from PIL import Image
import torch
import os

def compute_jpeg_residual_rgb(
    tensor_path,
    out_res_path,
    out_recon_png,
    out_vis_png,
    quality= 100,
    vis_clip= 32,
    save_results = True
):
    def jpeg_reconstruct_rgb(img_uint8: np.ndarray, quality: int = 100) -> np.ndarray:
        bio = BytesIO()
        Image.fromarray(img_uint8, mode="RGB").save(bio, format="JPEG", quality=quality, subsampling=0)
        bio.seek(0)
        rec = Image.open(bio).convert("RGB")
        return np.array(rec)

    def tensor_to_uint8_rgb(t: torch.Tensor) -> np.ndarray:
        if isinstance(t, torch.Tensor):
            t = t.detach().cpu()
        if t.dim() == 3 and t.shape[0] == 3:
            t = t.permute(1, 2, 0)
        arr = t.numpy()
        if arr.dtype in [np.float32, np.float64]:
            arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
        elif arr.dtype != np.uint8:
            arr = np.clip(arr, 0, 255).astype(np.uint8)
        return arr

    def uint8_to_tensor_rgb(arr_uint8: np.ndarray, normalize=True) -> torch.Tensor:
        t = torch.from_numpy(arr_uint8).permute(2, 0, 1).contiguous()
        return t.float() / 255.0 if normalize else t.float()

    def visualize_residual(residual_t: torch.Tensor, clip: int = 32) -> np.ndarray:
        res = residual_t.detach().cpu().numpy().astype(np.float32)
        res = np.transpose(res, (1, 2, 0))  # C,H,W → H,W,C
        res = np.clip(res, -clip, clip)
        res = (res + clip) / (2 * clip)
        vis = (res * 255.0).astype(np.uint8)
        return vis

    assert os.path.exists(tensor_path), f"Tensor not found: {tensor_path}"

    ext = os.path.splitext(tensor_path)[1].lower()
    if ext == ".pt":
        t = torch.load(tensor_path, map_location="cpu")
        assert isinstance(t, torch.Tensor), "Input .pt file must contain a torch.Tensor"
        assert t.shape[0] == 3, f"Expected RGB tensor (3,H,W), got {t.shape}"
    elif ext in [".jpg", ".jpeg", ".png"]:
        img = Image.open(tensor_path).convert("RGB")
        arr = np.array(img)
        t = torch.from_numpy(arr).permute(2, 0, 1).float() / 255.0
    else:
        raise ValueError(f"Unsupported file type: {tensor_path}")

    orig_uint8 = tensor_to_uint8_rgb(t)
    rec_uint8 = jpeg_reconstruct_rgb(orig_uint8, quality=quality)

    residual_i16 = orig_uint8.astype(np.int16) - rec_uint8.astype(np.int16)
    residual_t = torch.from_numpy(residual_i16).permute(2, 0, 1).contiguous().type(torch.int16)
    reconstructed_t = uint8_to_tensor_rgb(rec_uint8, normalize=True)

    vis = visualize_residual(residual_t, clip=vis_clip)
    vis_img = Image.fromarray(vis)

    res_np = residual_t.numpy()

    if save_results:
        base, _ = os.path.splitext(tensor_path)
        output_dir = os.path.dirname(out_res_path)
        os.makedirs(output_dir, exist_ok=True)
        torch.save(residual_t, out_res_path)
        vis_img.save(out_vis_png)
        Image.fromarray(rec_uint8).save(out_recon_png)

    return None

# Batch Function

In [15]:
import os
import pandas as pd
from tqdm import tqdm

def residual_batch(csv_path, base_input, base_output, img_size, data_type, quality = 75):
    df = pd.read_csv(csv_path)
    
    print(f"Loaded {len(df)} entries from {csv_path}")

    
    base_output = os.path.join(base_output, f'{img_size}x{img_size}', data_type)

    if data_type == 'clean':
        base_input = os.path.join(base_input, f'{img_size}x{img_size}')
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing JPGs"):
            rel_path = row["rel_path"].lstrip("/")  # remove leading slash if present
        
            in_path = os.path.join(base_input, rel_path)
            if not os.path.exists(in_path):
                print(f"Skipping missing image: {in_path}")
                continue
        
            rel_no_ext = os.path.splitext(rel_path)[0]
            out_res_path = os.path.join(base_output, f"{rel_no_ext}.pt")
            out_recon_png = os.path.join(base_output, f"{rel_no_ext}_reconstructed.png")
            out_vis_png = os.path.join(base_output, f"{rel_no_ext}_residual_vis.png")

            try:
                compute_jpeg_residual_rgb(
                    tensor_path=in_path,
                    out_res_path=out_res_path,
                    out_recon_png=out_recon_png,
                    out_vis_png=out_vis_png,
                    quality=quality,
                    vis_clip=32,
                    save_results=True
                )
            except Exception as e:
                print(f"Error processing {in_path}: {e}")
                
    else:
        base_input = os.path.join(base_input, f'{img_size}x{img_size}', data_type)
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing tensors"):
            rel_path = row["rel_path"].replace(".jpg", ".pt")
            
            in_path = os.path.join(base_input, rel_path.lstrip("/"))  # remove leading slash
            if not os.path.exists(in_path):
                print(f"Skipping missing tensor: {in_path}")
                continue
        
            rel_no_ext = os.path.splitext(rel_path.lstrip("/"))[0]
            out_res_path = os.path.join(base_output, f"{rel_no_ext}.pt")
            out_recon_png = os.path.join(base_output, f"{rel_no_ext}_reconstructed.png")
            out_vis_png = os.path.join(base_output, f"{rel_no_ext}_residual_vis.png")
    
            try:
                compute_jpeg_residual_rgb(
                    tensor_path=in_path,
                    out_res_path=out_res_path,
                    out_recon_png=out_recon_png,
                    out_vis_png=out_vis_png,
                    quality=quality,
                    vis_clip=32,
                    save_results=True
                )
            except Exception as e:
                print(f"Error processing {in_path}: {e}")



In [11]:
img_sizes = [32, 64, 128, 256, 512]
csv_path = "../02-AdvGenerate/Evaluate/common_success_details.csv"
base_input_adv = "../02-AdvGenerate/generated_images-test/"
base_input_clean = '../01-CleanModel/Dataset/'
base_output = "./jpg_residual-100/"

for img_size in img_sizes:
    data_types = ['clean', 'fgsm', 'bim', 'pgd', 'df', 'cw']
    for data_type in data_types:
        print(f'Extracting Residual ({img_size}x{img_size} - {data_type}) \n')
        if data_type == 'clean':
            base_input = base_input_clean
        else:
            base_input = base_input_adv
            
        residual_batch (csv_path, base_input, base_output, img_size, data_type, quality = 100)
    
        print ('Done. \n')

Extracting Residual (32x32 - clean) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


Processing JPGs: 100%|███████████████████████████████████████████████████████████████| 649/649 [00:21<00:00, 29.89it/s]


Done. 

Extracting Residual (32x32 - fgsm) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


Processing tensors: 100%|████████████████████████████████████████████████████████████| 649/649 [00:28<00:00, 22.67it/s]


Done. 

Extracting Residual (32x32 - bim) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


...

Done. 

Extracting Residual (512x512 - df) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


Processing tensors: 100%|████████████████████████████████████████████████████████████| 649/649 [04:47<00:00,  2.25it/s]


Done. 

Extracting Residual (512x512 - cw) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


Processing tensors: 100%|████████████████████████████████████████████████████████████| 649/649 [04:51<00:00,  2.22it/s]

Done. 



In [16]:
img_sizes = [1024]
csv_path = "../02-AdvGenerate/Evaluate/common_success_details.csv"
base_input_adv = "../02-AdvGenerate/generated_images-test/"
base_input_clean = '../01-CleanModel/Dataset/'
base_output = "./jpg_residual-100/"

for img_size in img_sizes:
    data_types = ['clean', 'fgsm', 'bim', 'pgd', 'df', 'cw']
    for data_type in data_types:
        print(f'Extracting Residual ({img_size}x{img_size} - {data_type}) \n')
        if data_type == 'clean':
            base_input = base_input_clean
        else:
            base_input = base_input_adv
            
        residual_batch (csv_path, base_input, base_output, img_size, data_type, quality = 100)
    
        print ('Done. \n')

Extracting Residual (1024x1024 - clean) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


Processing JPGs: 100%|███████████████████████████████████████████████████████████████| 649/649 [13:23<00:00,  1.24s/it]


Done. 

Extracting Residual (1024x1024 - fgsm) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


Processing tensors: 100%|████████████████████████████████████████████████████████████| 649/649 [18:52<00:00,  1.75s/it]


Done. 

Extracting Residual (1024x1024 - bim) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


...

Done. 

Extracting Residual (1024x1024 - df) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


Processing tensors: 100%|████████████████████████████████████████████████████████████| 649/649 [23:10<00:00,  2.14s/it]


Done. 

Extracting Residual (1024x1024 - cw) 

Loaded 649 entries from ../02-AdvGenerate/Evaluate/common_success_details.csv


Processing tensors: 100%|████████████████████████████████████████████████████████████| 649/649 [28:49<00:00,  2.67s/it]

Done. 

